In [12]:
import numpy as np
import pandas as pd

### 第四章 分组
#### 一. 分组模式及其对象
##### 1.分组的一般模式
分组操作在日常生活中使用极其广泛，例如：         
- 依据性别分组，统计全国人口寿命的平均值
- 依据季节分组，对每一个季节的温度进行组内标准化
- 依据班级筛选出组内数学分数的平均值超过80分的班级               
从上述的几个例子中不难看出，想要实现分组操作，必须明确三个要素：分组依据、数据来源、操作及其返回结果。    
同时从充分性的角度来说，如果明确了这三方面，就能确定一个分组操作，从而分组代码的一般模式即：

df.groupby(分组依据)[数据来源].使用操作

例如第一个例子中的代码就应该如下：

df.groupby('Gender')['Longevity'].mean()

现在返回到学生体测的数据集上，如果想要按照性别统计身高中位数，就可以如下写出：

In [13]:
df = pd.read_csv('../3_index/ch3/learn_pandas.csv')
df.groupby('Gender')['Height'].median()

Gender
Female    159.6
Male      173.4
Name: Height, dtype: float64

##### 2.分组依据的本质
前面提到的若干例子都是以单一维度进行分组的，比如根据性别，如果现在需要根据多个维度进行分组，该如何做？事实上，只需在groupby中传入相应列名构成的列表即可。例如，现希望根据学校和性别进行分组，统计身高的均值就可以如下写出：

In [14]:
df.groupby(['School', 'Gender'])['Height'].mean()

School  Gender
A       Female    159.122500
        Male      176.760000
B       Female    158.666667
        Male      172.030000
C       Female    158.776923
        Male      174.212500
D       Female    159.753333
        Male      171.638889
Name: Height, dtype: float64

目前为止，groupby的分组依据都是直接可以从列中按照名字获取的，那如果希望通过一定的复杂逻辑来分组，例如根据学生体重是否超过总体均值来分组，同样还是计算身高的均值。       
首先应该先写出分组条件：

In [15]:
condition = df.Weight > df.Weight.mean()

然后将其传入`groupby`中：

In [16]:
df.groupby(condition)['Height'].mean()

Weight
False    159.034646
True     172.705357
Name: Height, dtype: float64

练一练    
请根据上下四分位数分割，将体重分为high、normal、low三组，统计身高的均值.

In [ ]:
# q1 = df['Weight'].quantile(0.25)
# q3 = df['Weight'].quantile(0.75)
# def weight_group(weight):
#     if weight < q1:
#         return 'low'
#     elif weight > q3:
#         return 'high'
#     else:
#         return 'normal'
    
# df['WeightGroup'] = df['Weight'].apply(weight_group)
# df.groupby('WeightGroup')['Height'].mean()

WeightGroup
high      174.935714
low       153.753659
normal    162.177000
Name: Height, dtype: float64

从索引可以看出，其实最后产生的结果就是按照条件列表中元素的值（此处是True和False）来分组，下面用随机传入字母序列来验证这一想法：

In [17]:
item = np.random.choice(list('abc'), df.shape[0])
df.groupby(item)['Height'].mean()

a    161.764151
b    162.175362
c    165.660656
Name: Height, dtype: float64

此处的索引就是原先item中的元素，如果传入多个序列进入groupby，那么最后分组的依据就是这两个序列对应行的唯一组合：

In [18]:
df.groupby([condition, item])['Height'].mean()

Weight   
False   a    157.359459
        b    158.827451
        c    160.894872
True    a    171.950000
        b    171.661111
        c    174.109091
Name: Height, dtype: float64

由此可以看出，之前传入列名只是一种简便的记号，事实上等价于传入的是一个或多个列，最后分组的依据来自于数据来源组合的unique值，通过drop_duplicates就能知道具体的组类别：

In [19]:
df[['School', 'Gender']].drop_duplicates()

,School,Gender
0,A,Female
1,B,Male
2,A,Male
3,C,Female
4,C,Male
5,D,Female
9,B,Female
16,D,Male


In [20]:
df.groupby([df['School'],df['Gender']])['Height'].mean()

School  Gender
A       Female    159.122500
        Male      176.760000
B       Female    158.666667
        Male      172.030000
C       Female    158.776923
        Male      174.212500
D       Female    159.753333
        Male      171.638889
Name: Height, dtype: float64

##### 3.Groupby对象
能够注意到,最终具体做分组操作时，所调用的方法都来自于`pandas`中的`groupby`对象,这个对象上定义了许多方法，也具有一些方便的属性.

In [21]:
gb = df.groupby(['School', 'Grade'])
gb

通过`ngroups`属性，可以得到分组个数：

In [22]:
gb.ngroups

16

通过`groups`属性，可以返回从组名映射到组索引列表的字典:

In [25]:
res = gb.groups
res.keys() # 字典的值由于是索引，元素个数过多，此处只展示字典的键

dict_keys([('A', 'Freshman'), ('A', 'Junior'), ('A', 'Senior'), ('A', 'Sophomore'), ('B', 'Freshman'), ('B', 'Junior'), ('B', 'Senior'), ('B', 'Sophomore'), ('C', 'Freshman'), ('C', 'Junior'), ('C', 'Senior'), ('C', 'Sophomore'), ('D', 'Freshman'), ('D', 'Junior'), ('D', 'Senior'), ('D', 'Sophomore')])

当`size`作为`DataFrame`的属性时，返回的是表长乘以表宽的大小，但在`groupby`对象上表示统计每个组的元素个数：

In [26]:
gb.size()

School  Grade    
A       Freshman     13
        Junior       17
        Senior       22
        Sophomore     5
B       Freshman     13
        Junior        8
        Senior        8
        Sophomore     5
C       Freshman      9
        Junior       12
        Senior       11
        Sophomore     8
D       Freshman     17
        Junior       22
        Senior       14
        Sophomore    16
dtype: int64

通过`get_group`方法可以直接获取所在组对应的行，此时必须知道组的具体名字：

In [27]:
gb.get_group(('A','Freshman'))

,School,Grade,Name,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
0,A,Freshman,Gaopeng Yang,Female,158.9,46.0,N,1,2019/10/5,0:04:34
6,A,Freshman,Qiang Chu,Female,162.5,52.0,N,1,2019/12/12,0:03:53
10,A,Freshman,Xiaopeng Zhou,Male,174.1,74.0,N,1,2019/9/29,0:05:16
60,A,Freshman,Yanpeng Lv,Male,NaN,65.0,N,1,2019/11/17,0:04:13
114,A,Freshman,Xiaopeng Zhao,Female,161.0,53.0,N,3,2019/9/25,0:05:13
117,A,Freshman,Chunli Zhao,Male,180.2,83.0,N,1,2020/1/7,0:04:33
119,A,Freshman,Peng Zhang,Female,163.1,NaN,N,3,2019/9/23,0:04:31
121,A,Freshman,Xiaoquan Sun,Female,154.6,40.0,N,3,2019/11/12,0:04:05
141,A,Freshman,Chunmei Shi,Female,164.9,52.0,N,1,2019/9/8,0:03:33
148,A,Freshman,Xiaomei Yang,Female,159.3,49.0,N,1,2019/9/17,0:04:22


这里列出了2个属性和2个方法，而先前的mean、median都是groupby对象上的方法，这些函数和许多其他函数的操作具有高度相似性，将在之后的小节进行专门介绍。

##### 4.分组的三大操作
熟悉了一些分组的基本知识后，重新回到开头举的三个例子，可能会发现一些端倪，即这三种类型分组返回的数据型态并不一样：       
- 第一个例子中，每一个组返回一个标量值，可以是平均值，中位数，组容量`size`等
- 第二个例子中，做了原序列的标准化处理，也就是说每组返回的是一个`Series`类型
- 第三个例子中，既不是标量也不是序列，返回的整个组所在行的本身，即返回了`DataFrame`类型         
由此，引申出分组的三大操作：聚合、变换和过滤，分别对应了三个例子的操作，下面就要分别介绍相应的`agg`、`transform`和`filter`函数及其操作